## Demo of the workflow
- create tiles of images
- find tiles in other notebooks
- build a stack of tiles
- assess image quality in stack

In [ ]:
import os

import pandas as pd
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from pathlib import Path

full_image = "./tests/data/DJI_0018.JPG"

image = Image.open(full_image)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

imr = np.array(image, dtype=np.uint8)
ax.imshow(imr)
ax.set_title("original image")

In [ ]:
### Create Tiles

In [ ]:
from image_template_search.image_rasterization import create_regular_raster_grid

crop_height = crop_width = 1024
grid, _ = create_regular_raster_grid(max_x=image.size[0], max_y=image.size[1],
                                                     slice_height=crop_height, slice_width=crop_width)

In [ ]:

from image_template_search.image_rasterization import crop_polygons

crops = crop_polygons(image, grid)

for i, crop in enumerate(crops):
    crop.save(f"tests/output/crops/crop_{Path(full_image).stem}_{i}.jpg")
    print(f"saved crop_{i}.jpg")

### Looking for the patches

In [ ]:
from image_template_search.image_similarity import find_patch

## Find a crop within other images

template_name = "crop_DJI_0018_7.jpg"
template_path = Path(f"tests/output/crops/{template_name}")
large_image_path = Path(full_image)

crop = find_patch(template_path, large_image_path, output_path="tests/output")


#### For the same patch by tiling the large image
Can be useful for large images like orthomosaics

In [ ]:
import tempfile
from image_template_search.image_similarity import find_patch_tiled

with tempfile.TemporaryDirectory() as cache_dir:
    print('created temporary directory', cache_dir)

    cache_dir = Path(cache_dir)

    crop = find_patch_tiled(template_path, large_image_path,
                            output_path=Path("tests/output"),
                            tile_size_x=1200,
                            tile_size_y=1200, 
                            cache_path=None)

crop